# Preprocessing
1. Convert text to lowercase 
2. Remove URLs, mentions, and special characters (besides hashtags and emojis)
3. Remove stop words
4. Perform stemming/lemmatization

Download the spaCy model
Can be installed via `python -m spacy download en_core_web_sm`

In [7]:
import pandas as pd
import re
import spacy
import emoji

In [8]:
# Load the JSON file
df_posts = pd.read_json('../dataset.json')
df_posts.head()

,timestamp,text,text_id,user,user_id
0,2024-10-31,Running a business means juggling countless ad...,2018569761,danielwoodard,1077866112
1,2024-10-31,Liz Truss is walking in the lingering shadow o...,2092717718,nelsonjacqueline,1089670430
2,2024-10-31,The UK is bracing for war as government buildi...,2059143248,ihooper,1007478642
3,2024-10-31,Marrying a second or third cousin once removed...,2008209828,wrightnicholas,1039258480
4,2024-10-31,It's truly disgraceful how the Indian National...,2001239278,michael51,1021455936


In [9]:
spacy.prefer_gpu()  # Prefers GPU but doesn't crash if unavailable
nlp = spacy.load("en_core_web_trf")
def extract_entities(text):
    """
    Extracts named entities from text using SpaCy's NER model.

    Args:
    text (str): The text from which to extract named entities.

    Returns:
    list: A list of tuples where each tuple contains (entity_text, entity_label).
    """
    if not text or pd.isna(text):
        return []  # Return an empty list if text is missing

    # Process text with SpaCy
    doc = nlp(text)

    # Extract entity text and labels
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    return entities

def preprocess_text(text):
    """
    Preprocesses text by removing URLs and emojis while keeping mentions and hashtags intact.

    Args:
    text (str): The original text.

    Returns:
    str: Preprocessed text.
    """
    if not text or pd.isna(text):
        return ""  # Return empty string if text is missing

    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)
    # Remove emojis
    text = emoji.replace_emoji(text, replace="")
    
    return text.strip()

# Create a preprocessed text column
df_posts['preprocessed_text'] = df_posts['text'].apply(preprocess_text)

# Apply NER extraction on the preprocessed text
df_posts['entities'] = df_posts['preprocessed_text'].apply(extract_entities)

/Users/lauritseisengarten/Documents/GitHub/case-study-ul/venv/lib/python3.12/site-packages/thinc/shims/pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.lo

Check for rows with no text

In [10]:
# Display rows where 'text' is missing (NaN)
missing_text_rows = df_posts[df_posts['text'].isnull()]
missing_text_rows

,timestamp,text,text_id,user,user_id,preprocessed_text,entities


### Convert variables

Remove Hashtags to preserve casings

In [11]:
# Apply re.findall() to each row in the 'text' column to extract hashtags
df_posts['hashtags'] = df_posts['text'].apply(lambda x: re.findall(r'#\w+', x) if isinstance(x, str) else [])

Convert text to lowercase

In [12]:
df_posts['text'] = df_posts['text'].str.lower()

Remove Date from Timestamp

In [13]:
# All Dates are the same
unique_dates = df_posts['timestamp'].dt.date.unique()
unique_dates

array([datetime.date(2024, 10, 31)], dtype=object)

In [14]:
df_posts['timestamp'] = df_posts['timestamp'].dt.time
df_posts.head()

,timestamp,text,text_id,user,user_id,preprocessed_text,entities,hashtags
0,00:00:00,running a business means juggling countless ad...,2018569761,danielwoodard,1077866112,Running a business means juggling countless ad...,[],"[#HRtech, #businessmanagement]"
1,00:00:00,liz truss is walking in the lingering shadow o...,2092717718,nelsonjacqueline,1089670430,Liz Truss is walking in the lingering shadow o...,"[(Liz Truss, PERSON)]",[#politics]
2,00:00:00,the uk is bracing for war as government buildi...,2059143248,ihooper,1007478642,The UK is bracing for war as government buildi...,"[(UK, GPE), (London, GPE), (Ukraine, GPE), (Mi...","[#Ukrainewashed, #WarPreparedness]"
3,00:00:00,marrying a second or third cousin once removed...,2008209828,wrightnicholas,1039258480,Marrying a second or third cousin once removed...,"[(second, ORDINAL), (third, ORDINAL)]","[#FamilyTree, #GeneticFacts]"
4,00:00:00,it's truly disgraceful how the indian national...,2001239278,michael51,1021455936,It's truly disgraceful how the Indian National...,"[(the Indian National Congress, ORG)]",[#RationChorCongress]


### Remove URLs, Mentions, and Special Characters

In [15]:
# Pre-compile regex patterns
emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags
    u"\U00002700-\U000027BF"  # dingbats
    u"\U0001F900-\U0001F9FF"  # supplemental symbols and pictographs
    u"\U00002600-\U000026FF"  # miscellaneous symbols
    u"\U00002B50-\U00002B55"  # stars
    "]+", flags=re.UNICODE)

url_pattern = re.compile(r'http\S+|www\S+|https\S+')
mention_pattern = re.compile(r'@(\w+)')  # Removes "@" but keeps the mention
punctuation_pattern = re.compile(r'[^\w\s]')  # Removes punctuation
number_pattern = re.compile(r'\d+')  # Removes numbers
whitespace_pattern = re.compile(r'\s+')  # Removes excessive whitespace
hashtag_pattern = re.compile(r'#\w+.*')  # Removes hashtags and all text after them

# Optimized function
def preprocess_text(text):
    if pd.isna(text):
        return "", []  # Handle missing values gracefully

    # Extract emojis
    emojis = emoji_pattern.findall(text)  # List of emojis

    # Remove hashtags and text following them
    text = hashtag_pattern.sub('', text)

    # Remove emojis, URLs, mentions, punctuation, and numbers
    text = emoji_pattern.sub('', text)  # Remove emojis
    text = url_pattern.sub('', text)  # Remove URLs
    text = mention_pattern.sub(r'\1', text)  # Remove "@" but keep mention names
    text = punctuation_pattern.sub('', text)  # Remove punctuation
    text = number_pattern.sub('', text)  # Remove numbers

    # Remove excessive whitespace and trim
    text = whitespace_pattern.sub(' ', text).strip()

    return text, emojis

# Apply preprocessing to create new columns
df_posts[['text', 'emojis']] = df_posts['text'].apply(lambda x: pd.Series(preprocess_text(x)))

# Display head to check the results
df_posts[['text', 'emojis']].head()

,text,emojis
0,running a business means juggling countless ad...,[]
1,liz truss is walking in the lingering shadow o...,[]
2,the uk is bracing for war as government buildi...,[🇺🇦]
3,marrying a second or third cousin once removed...,[🧬]
4,its truly disgraceful how the indian national ...,"[🤦, ♂]"


## Remove stopwords

In [16]:
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"]) # disable ner and parser as these are not needed for stop word removal and lemmatizer

In [17]:
def remove_stopwords_fast(text):
    if pd.isna(text):
        return ""
    doc = nlp.make_doc(text)  # Only tokenization
    filtered_text = ' '.join([token.text for token in doc if not token.is_stop])
    return filtered_text

# Apply the function to the 'text' column of the DataFrame
df_posts['text'] = df_posts['text'].apply(remove_stopwords_fast)

# Lemmatization

Use lemmatization since stemming can lead to less accurate results (even non-words)

In [19]:
nlp = spacy.load("en_core_web_sm")
def lemmatize_text_spacy(text):
    if pd.isna(text):  # Handle missing values
        return ""
    doc = nlp(text)  # Process the text using spaCy
    lemmatized_text = ' '.join([token.lemma_ for token in doc if not token.is_stop])  # Lemmatize each word
    return lemmatized_text

# Apply lemmatization to the 'text' column
df_posts['text'] = df_posts['text'].apply(lemmatize_text_spacy)

In [22]:
df_posts

,timestamp,text,text_id,user,user_id,preprocessed_text,entities,hashtags,emojis
0,00:00:00,run business mean juggle countless administrat...,2018569761,danielwoodard,1077866112,Running a business means juggling countless ad...,[],"[#HRtech, #businessmanagement]",[]
1,00:00:00,liz truss walk linger shadow predecessor charl...,2092717718,nelsonjacqueline,1089670430,Liz Truss is walking in the lingering shadow o...,"[(Liz Truss, PERSON)]",[#politics],[]
2,00:00:00,uk brace war government building london raise ...,2059143248,ihooper,1007478642,The UK is bracing for war as government buildi...,"[(UK, GPE), (London, GPE), (Ukraine, GPE), (Mi...","[#Ukrainewashed, #WarPreparedness]",[🇺🇦]
3,00:00:00,marry second cousin remove not taboo think gen...,2008209828,wrightnicholas,1039258480,Marrying a second or third cousin once removed...,"[(second, ORDINAL), (third, ORDINAL)]","[#FamilyTree, #GeneticFacts]",[🧬]
4,00:00:00,truly disgraceful indian national congress sto...,2001239278,michael51,1021455936,It's truly disgraceful how the Indian National...,"[(the Indian National Congress, ORG)]",[#RationChorCongress],"[🤦, ♂]"
...,...,...,...,...,...,...,...,...,...
70255,23:59:52,bakerjulie saranghae m resident kpop lover gra...,2086649509,valdezjennifer,1094330726,"@bakerjulie: ""Saranghae, I’m your resident K-P...","[(@bakerjulie, PERSON), (Saranghae, PERSON)]","[#KPopArt, #StayCreative]","[🌸✨, 💖🎨]"
70256,23:59:52,soar new height sweeneyanthony future look bright,2039889186,nashshaun,1015245531,Soaring to new heights with @sweeneyanthony! T...,"[(@sweeneyanthony, ORG)]","[#caesarstonesa, #designinspiration, #modernin...","[✨, 🚀]"
70257,23:59:54,hey james paul late axa disclaimer shady test ...,2020468196,brownregina,1029384492,"Hey @james20 @paul47, did you see the latest f...","[(AXA, ORG), (AXA, ORG)]","[#AXA, #truthprevails]",[]
70258,23:59:58,check fundraiser exhibition joanna vbrewer sep...,2037744299,dkey,1046050046,Check out the fundraiser exhibition by @joanna...,"[(@joanna37, PERSON), (@vbrewer, ORG), (Sept 1...","[#artandmentalhealth, #hauserwirth, #hospitalr...",[]


### Write the output to csv file

In [21]:
df_posts.to_csv('../output/preprocessed.csv', index=False)